## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

In [2]:
import string
from collections import Counter # Conta as palavras no dataset
import re
from typing import List, Dict, Union
import random

import numba
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



In [3]:
random.seed(18)

## Faz download e carrega o dataset

In [ ]:
!wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
!wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [9]:
!curl -LO https://www.gutenberg.org/ebooks/67724.txt.utf-8
!curl -LO https://www.gutenberg.org/ebooks/67725.txt.utf-8

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   304  100   304    0     0    556      0 --:--:-- --:--:-- --:--:--   558

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

 11  364k   11 42594    0     0  34836      0  0:00:10  0:00:01  0:00:09 34836
100  364k  100  364k    0     0   194k      0  0:00:01  0:00:01 --:--:--  495k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   304  100   304    0     0    556      0 --

In [4]:
text = open("67724.txt.utf-8","r", encoding="utf8").read()
text += open("67725.txt.utf-8","r", encoding="utf8").read()

paragraphs = text.split("\n\n")
len(paragraphs)

4969

In [5]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

#Paper:
#ponctuation -> keep (separado das outras palavras, "pontuação," -> "pontuação"+",")
#numeric -> special symbol (colocando todos como 999 para convergir para o mesmo símbolo)
#upper -> lower
#proper nouns -> special symbol (difícil identificar, ignorado)
#rare words -> special symbol (feito na parte de encoding)

cleaned_paragraphs = [cleaned_paragraph.lower() for cleaned_paragraph in cleaned_paragraphs]

number_counter = Counter()


for i in range(len(cleaned_paragraphs)):
    old_p = cleaned_paragraphs[i].split()
    new_p = []

    for j in range(len(old_p)):
        word = old_p[j] 
        if word.isdigit():
            number_counter.update(word)
            word = "999"
        elif len(word) > 1 and word[0] in string.punctuation:
            old_p.insert(j+1, word[1:])
            word = word[0]
        elif word[-1] in string.punctuation and len(word) > 1:
            old_p.insert(j+1, word[:-1])
            old_p.insert(j+2, word[-1])
            
            word = ""
        
        if len(word) > 0:
            new_p.append(word)
        #j += 1
    
    cleaned_paragraphs[i] = " ".join(new_p)

print("SAMPLE ----------------")
print(cleaned_paragraphs[0])
print("---------------------")

print(len(cleaned_paragraphs))



SAMPLE ----------------
﻿the project gutenberg ebook of o guarany : romance brazileiro , vol . 999 ( of 999 ) this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever . you may copy it , give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org . if you are not located in the united states , you
---------------------
4892


In [6]:
number_counter

Counter({'1': 37,
         '2': 13,
         '8': 23,
         '7': 7,
         '3': 11,
         '5': 23,
         '6': 8,
         '0': 31,
         '4': 7,
         '9': 10})

In [7]:
del paragraphs, number_counter, new_p, old_p, text

## Análise do dataset

In [8]:

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        #Regular expression removes ponctuation
        #word_counts.update(re.findall(r'\w+', text.lower())) 
        word_counts.update(text.split(" "))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

11470

## Criando um vocabulário

In [9]:
vocab_size = 10000 #3000#
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [10]:
def encode_sentence(sentence:Union[str,List[str]], vocab:Dict) -> List[int]:
    if isinstance(sentence, list):
        words = sentence
    else:
        words = sentence.split(" ") #Removido o regex por não pegar pontuação e ser ~3x mais lento
    
    return [vocab.get(word, 0) for word in words]

Checando se o encoding faz sentido.

As palavras mais frequentes são pontuações, potencialmente problemático.

In [11]:
print("20 palavras mais frequentes:", most_frequent_words[:20])

20 palavras mais frequentes: [',', 'a', 'que', '-', 'o', 'de', 'e', ';', '.', 'um', 'do', 'não', 'uma', 'os', 'se', 'da', 'com', 'sua', 'para', 'seu']


In [13]:
encoded20 = encode_sentence(cleaned_paragraphs[20], vocab)
words = cleaned_paragraphs[20].split(" ")

for i in range(len(words)):
    print(words[i], encoded20[i])

publicando 5496
este 126
livro 3463
em 21
999 153
, 1
se 15
disse 57
ser 122
aquella 221
primeira 197
edição 2103
uma 13
prova 960
typographica 5497
, 1
que 3
algum 192
dia 134
talvez 281
o 5
autor 2105


In [14]:
del word_counts, most_frequent_words, encoded20, words

NameError: name 'most_frequent_words' is not defined

## Classe do dataset

In [15]:
context_size = 5 # 5 palavras de entrada. O target é a próxima palavra

x_all = []
y_all = []

for paragraph in cleaned_paragraphs:
    start = 0
    end = context_size

    paragraph = encode_sentence(paragraph, vocab)

    while end < len(paragraph):
        x = paragraph[start:end]
        y = paragraph[end]

        if not ( 0 in x or 0 == y):
            x_all.append(x)
            y_all.append(y)

        start += 1
        end += 1

In [16]:
del paragraph

Checando se o dataset está correto

In [17]:
print(encode_sentence(cleaned_paragraphs[0], vocab))

for i in range(3):
    print(x_all[i], "|", y_all[i])

[3440, 68, 214, 493, 48, 5, 684, 44, 866, 867, 1, 3441, 9, 153, 348, 48, 153, 383, 145, 493, 309, 262, 37, 494, 48, 957, 2095, 106, 37, 411, 329, 92, 958, 412, 2096, 48, 37, 2097, 645, 24, 1492, 92, 136, 2098, 24, 2099, 2100, 9, 82, 446, 495, 384, 1, 1493, 384, 2101, 93, 2102, 384, 802, 37, 276, 48, 37, 68, 214, 447, 1494, 136, 145, 493, 93, 1495, 645, 1496, 9, 263, 82, 252, 215, 803, 106, 37, 411, 329, 1, 82]
[3440, 68, 214, 493, 48] | 5
[68, 214, 493, 48, 5] | 684
[214, 493, 48, 5, 684] | 44


In [18]:
len(x_all)

74741

In [19]:
assert len(x_all) == len(y_all)

Divisão treino|validação|teste

60%|20%|20%

OBS: seed determinada no início do notebook

In [20]:
#Embaralhando para evitar viés
indexes = list(range(len(x_all)))
random.shuffle(indexes)

x_all = np.array(x_all)
y_all = np.array(y_all)

x_all = x_all[indexes]
y_all = y_all[indexes]

In [21]:
size_all = len(x_all)

cut1 = int(0.6*size_all)
cut2 = int(0.8*size_all)

x_train = x_all[0:cut1]
y_train = y_all[0:cut1]

x_val = x_all[cut1:cut2]
y_val = y_all[cut1:cut2]

x_test = x_all[cut2:]
y_test = y_all[cut2:]

In [22]:
assert len(x_train)+len(x_val)+len(x_test) == size_all

Classe para o dataset

OBS: utilizar tensores q [context_size x vocab_size] esparsos utiliza muita memória, preferi alterar a primeira camada para evitar precisar gerar estes tensores

In [23]:
class TextPredictDataset(Dataset):
    def __init__(self, x_data:List[int], y_data:List[int]):
        self._x_data = torch.tensor(x_data)
        self._y_data = torch.tensor(y_data)
        
        if len(x_data) != len(y_data):
            raise ValueError(f"x_data and y_data must have same size. ({len(x_data)} ≠ {len(y_data)})")
        
        self._size = len(x_data)

    def __len__(self):
        return self._size

    def __getitem__(self, idx):
        return self._x_data[idx], self._y_data[idx]


In [28]:
train_data = TextPredictDataset(x_train, y_train)
val_data = TextPredictDataset(x_val, y_val)
test_data = TextPredictDataset(x_test, y_test)

In [29]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [31]:
sample_batch = next(iter(train_loader))

## Model

In [118]:

class LanguageModel(nn.Module):
    """TODO: implementar o modelo de linguagem"""

    def __init__(self, context_size:int, vocab_size:int, embed_dim:int, hidden_units:int):
        super().__init__()

        self.C = torch.Tensor(vocab_size, embed_dim)
        nn.init.xavier_uniform_(self.C)
        self.C = torch.nn.Parameter(self.C)
        
        
        #V = |Vocab|, m = |Embed|
        #n-1 = c = |Context|
        #h = |Hidden|
        
        #C[V, m](input) -> x[c*m]
        #Linear1(x) -> x2[h]
        #ReLU(x2) -> x3[h]  | alterado do paper (tanh)
        #Linear2(x) -> x4[V]
        #Linear3(x3) -> x5[V] | sem bias (Linear2 já tem bias)
        #Add(x4, x5) -> output
        #Sem softmax -> melhor estabilidade

        m = int(context_size*embed_dim)

        self.linear1 = nn.Linear(m, hidden_units)
        self.relu = nn.ReLU() 
        self.linear2 = nn.Linear(m, vocab_size)
        self.linear3 = nn.Linear(hidden_units, vocab_size, bias=False)

    def forward(self, input_x:torch.Tensor) -> torch.Tensor:
        #No batch: x = torch.index_select(self.C, 0, input_x).flatten()
        x = torch.stack([torch.index_select(self.C, 0, input_i).flatten() for input_i in input_x])
        
        x2 = self.linear1(x)
        x3 = self.relu(x2)
        x4 = self.linear2(x)
        x5 = self.linear3(x3)

        output = x4+x5

        return output

In [132]:
embed_dim = 64
hidden_units = 300
model = LanguageModel(context_size, vocab_size, embed_dim, hidden_units)

In [133]:
# sample = next(iter(train_loader))
input = sample_batch[0]
target = sample_batch[1]

In [134]:
output = model(input)

In [135]:
output.argmax(dim=1)

tensor([2656, 2773, 9901, 9799, 2881, 3629, 3690, 2576, 6445,  515, 3629, 9799,
        7575, 1506, 9901, 7575, 6302, 2656, 2881, 6445, 6445, 6445, 7474, 8238,
         515, 3629, 6721, 9799, 5800, 6302, 9799, 9901])

In [136]:
target

tensor([   7,    8,   65,  112,  869,  238,    1,  144,  176,  164,   22,    3,
           1, 4044,    5,  117,    2,   24, 1084,  676,  394,    2,   14,  817,
         108,    1,    7, 2807,    6,    8,  908,    1], dtype=torch.int32)

In [137]:
assert output.argmax(dim=1).shape == target.shape

In [138]:
 #V = |Vocab|, m = |Embed|
        #n-1 = c = |Context|
        #h = |Hidden|

n_param_real = sum([p.numel() for p in model.parameters()])

m = embed_dim
n = context_size+1
V = vocab_size
h = hidden_units

n_param_theoretical = V*(1+(n*m)+h)
n_param_theoretical += h*(1+((n-1)*m))

assert n_param_real == n_param_theoretical

In [139]:
print(n_param_real, "parâmetros")

6946300 parâmetros


## Training

In [140]:
# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
epochs = 10
lr = """TODO""""
criterion = """TODO CrossEntropy""""

optimizer = """TODO: AdamW ou outro""""

model.to(device)

"""TODO: Implemente o loop de treinamento. Em cada época, calcule e imprima a loss no dataset de validação""""

## Avaliação

In [ ]:
""" TODO: calcule a perplexidade final no dataset de validação """

## Exemplo de uso

In [ ]:
text = ""

def generate_text(model, vocab, text, max_length):
    """TODO: implemente a função para gerar texto até atingir o max_length"""

context = 5
max_length= 10
generate_text(text, max_length)